In [1]:
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
# 강아지 
import json

base_url= 'https://www.pimfyvirus.com/search/01_v/{}/'
base_img_url = 'https://www.pimfyvirus.com'
# print(type(base_url))

req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

info_keys = ['현 상황','임보종류','이름', '구조지역', '성별','중성화여부','출생시기','몸무게']

with open("data/dog_id.txt", "r", encoding="utf-8") as f:
    dog_ids = [line.strip() for line in f if line.strip()]   

dog_info_list = []    
for dog_id in dog_ids:
    dog_info_dict = dict() 
    url = base_url.format(dog_id)
    res = requests.get(url, headers=req_header)

    if res.ok:
        soup = BeautifulSoup(res.text, "html.parser")

        #이미지 
        img = soup.select_one('button.img img[src]')
        full_img = base_img_url + img.get('src')
        dog_info_dict["이미지"] = full_img
        print(full_img)
        
        #이름,성별,나이등 info_keys 정보(첫번째 table)  
        tags = []
        table = soup.select_one('div.tb_wrap table')
        if table:
            for row in table.find_all("tr"):
                th = row.find("th")
                td = row.find("td")
                if th and td:
                    key = th.text.strip()
                    value = td.text.strip()
                    if key in ['현 상황', '임보종류']:
                        tags.append(value)  # 태그 리스트에 추가
                    elif key in info_keys:
                        dog_info_dict[key] = value
        dog_info_dict["태그"] = tags
        print("태그:", tags)
        print("이름:", dog_info_dict.get("이름"))

        #성격 및 특징 설명 
        p_list = soup.select('div.box > p.txt')
        if p_list:
            p = p_list[1]  # 두 번째 <p class="txt">
            for br in p.find_all('br'):
                br.replace_with('\n')  

            feature = p.text.strip()
            dog_info_dict['성격 및 특징'] = feature
            # print(f'{feature}')

        
        #임보조건
        for p_tag in soup.select('p.tit'):
            if '임보 조건' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    care_conditions = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        key = th.text.strip()
                        value = td.text.strip()
                        if key not in ['픽업', '기타 조건']:  # ← 이 줄이 핵심
                            care_conditions[key] = value
                    dog_info_dict["임보 조건"] = care_conditions
                    # print(care_conditions)

        #히스토리
        for p_tag in soup.select('p.tit'):
            if '히스토리' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    history = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        if th and td:
                            history[th.text.strip()] = td.text.strip()
                    dog_info_dict["히스토리"] = history
                    # print(history)

        #건강정보 
        for p_tag in soup.select('p.tit'):
            if '건강 정보' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    health_info = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        if th and td:
                            health_info[th.text.strip()] = td.text.strip()
                    dog_info_dict["건강 정보"] = health_info
                    # print(health_info)

        #공고날짜
        p_tags = soup.select_one('div.info p.date.c9')
        p_tag = p_tags.text.strip()
        dog_info_dict["공고날짜"] = p_tag
        # print(f'{p_tag}')

        #사이트링크
        dog_info_dict["사이트링크"] = url
        # print(url)
   
        dog_info_list.append(dog_info_dict)

        

    else:
        print(f"[{dog_id}] 요청 실패: {res.status_code}")

with open('data/dog_info.json', 'w', encoding='utf-8') as f:
    json.dump(dog_info_list, f, ensure_ascii=False, indent=2)


https://www.pimfyvirus.com/inday_fileinfo/img/20250731174533.jpg
태그: ['임보가능', '긴급임보']
이름: [댕잇슈] 로제
https://www.pimfyvirus.com/inday_fileinfo/img/20250731163634.jpg
태그: ['임보가능', '릴레이임보']
이름: 엘리
https://www.pimfyvirus.com/inday_fileinfo/img/20250731160907.jpg
태그: ['임보가능', '일반임보']
이름: 단비
https://www.pimfyvirus.com/inday_fileinfo/img/20250730184056.jpg
태그: ['임보가능', '입양전제']
이름: 니콜
https://www.pimfyvirus.com/inday_fileinfo/img/20250730183117.jpg
태그: ['임보가능', '단기임보']
이름: 나오미
https://www.pimfyvirus.com/inday_fileinfo/img/20250730145831.jpg
태그: ['임보중', '단기임보']
이름: 윈터
https://www.pimfyvirus.com/inday_fileinfo/img/20250729221729.jpg
태그: ['임보가능', '일반임보']
이름: [함안길천사]다나
https://www.pimfyvirus.com/inday_fileinfo/img/20250729221416.jpg
태그: ['임보가능', '일반임보']
이름: [함안길천사]다온
https://www.pimfyvirus.com/inday_fileinfo/img/20250729221110.jpg
태그: ['임보가능', '일반임보']
이름: [함안길천사]다루
https://www.pimfyvirus.com/inday_fileinfo/img/20250729220514.jpg
태그: ['임보가능', '일반임보']
이름: [함안길천사]다솔
https://www.pimfyvirus.com/inday_fi

In [3]:
# 고양이
import json

base_url= 'https://www.pimfyvirus.com/search/02_v/{}/'
base_img_url = 'https://www.pimfyvirus.com'
# print(type(base_url))

req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

info_keys = ['현 상황','임보종류','이름', '구조지역', '성별','중성화여부','출생시기','몸무게']

with open("data/cat_id.txt", "r", encoding="utf-8") as f:
    dog_ids = [line.strip() for line in f if line.strip()]   

dog_info_list = []    
for dog_id in dog_ids:
    dog_info_dict = dict() 
    url = base_url.format(dog_id)
    res = requests.get(url, headers=req_header)

    if res.ok:
        soup = BeautifulSoup(res.text, "html.parser")

        #이미지 
        img = soup.select_one('button.img img[src]')
        full_img = base_img_url + img.get('src')
        dog_info_dict["이미지"] = full_img
        print(full_img)
        
        #이름,성별,나이등 info_keys 정보(첫번째 table)  
        tags = []
        table = soup.select_one('div.tb_wrap table')
        if table:
            for row in table.find_all("tr"):
                th = row.find("th")
                td = row.find("td")
                if th and td:
                    key = th.text.strip()
                    value = td.text.strip()
                    if key in ['현 상황', '임보종류']:
                        tags.append(value)  # 태그 리스트에 추가
                    elif key in info_keys:
                        dog_info_dict[key] = value
        dog_info_dict["태그"] = tags
        print("태그:", tags)
        print("이름:", dog_info_dict.get("이름"))

        #성격 및 특징 설명 
        p_list = soup.select('div.box > p.txt')
        if p_list:
            p = p_list[1]  # 두 번째 <p class="txt">
            for br in p.find_all('br'):
                br.replace_with('\n')  

            feature = p.text.strip()
            dog_info_dict['성격 및 특징'] = feature
            # print(f'{feature}')

        
        #임보조건
        for p_tag in soup.select('p.tit'):
            if '임보 조건' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    care_conditions = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        key = th.text.strip()
                        value = td.text.strip()
                        if key not in ['픽업', '기타 조건']:  # ← 이 줄이 핵심
                            care_conditions[key] = value
                    dog_info_dict["임보 조건"] = care_conditions
                    # print(care_conditions)

        #히스토리
        for p_tag in soup.select('p.tit'):
            if '히스토리' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    history = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        if th and td:
                            history[th.text.strip()] = td.text.strip()
                    dog_info_dict["히스토리"] = history
                    # print(history)

        #건강정보 
        for p_tag in soup.select('p.tit'):
            if '건강 정보' in p_tag.get_text():
                target_div = p_tag.find_next_sibling('div', class_='tb_wrap')
                if target_div:
                    table = target_div.find('table')
                    rows = table.select('tr')
                    health_info = {}
                    for row in rows:
                        th = row.find('th')
                        td = row.find('td')
                        if th and td:
                            health_info[th.text.strip()] = td.text.strip()
                    dog_info_dict["건강 정보"] = health_info
                    # print(health_info)

        #공고날짜
        p_tags = soup.select_one('div.info p.date.c9')
        p_tag = p_tags.text.strip()
        dog_info_dict["공고날짜"] = p_tag
        # print(f'{p_tag}')

        #사이트링크
        dog_info_dict["사이트링크"] = url
        # print(url)
   
        dog_info_list.append(dog_info_dict)

        

    else:
        print(f"[{dog_id}] 요청 실패: {res.status_code}")

with open('data/cat_info.json', 'w', encoding='utf-8') as f:
    json.dump(dog_info_list, f, ensure_ascii=False, indent=2)

https://www.pimfyvirus.com/inday_fileinfo/img/20250726192720.jpg
태그: ['임보가능', '입양전제']
이름: 나기 외 새끼고양이
https://www.pimfyvirus.com/inday_fileinfo/img/20250721212529.jpg
태그: ['임보가능', '입양전제']
이름: 달봉이
https://www.pimfyvirus.com/inday_fileinfo/img/20250716234832.jpg
태그: ['임보중', '입양전제']
이름: 콩이
https://www.pimfyvirus.com/inday_fileinfo/img/20250715190031.jpg
태그: ['임보중', '입양전제']
이름: 로담
https://www.pimfyvirus.com/inday_fileinfo/img/20250710164113.jpg
태그: ['임보중', '입양전제']
이름: 로아
https://www.pimfyvirus.com/inday_fileinfo/img/20250709214505.jpg
태그: ['임보가능', '입양전제']
이름: 흑미
https://www.pimfyvirus.com/inday_fileinfo/img/20250708233120.jpeg
태그: ['임보중', '입양전제']
이름: 방울이
https://www.pimfyvirus.com/inday_fileinfo/img/20250708232300.jpeg
태그: ['임보중', '입양전제']
이름: 초롱이
https://www.pimfyvirus.com/inday_fileinfo/img/20250708163238.jpg
태그: ['임보중', '일반임보']
이름: 초롱이
https://www.pimfyvirus.com/inday_fileinfo/img/20250708143121.jpg
태그: ['임보중', '입양전제']
이름: 강이
https://www.pimfyvirus.com/inday_fileinfo/img/20250708141620.jp